# 1. Вытаскиваем ссылки, которые накачали

In [2]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

# библиотека для создания юзерагентов
from fake_useragent import UserAgent

from tqdm import tqdm_notebook
import re
import pickle
import time
 
# модуль с мелкими функциями
from cian_info import *

In [3]:
with open('hrefs_total.pickle', 'rb') as f:
    hrefs_total = pickle.load(f)
    
len(hrefs_total)

73924

In [4]:
type(hrefs_total), len(set(hrefs_total))

(list, 73924)

### Чтобы включать ТОР

In [5]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

In [6]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 77.247.181.162


# 2. Качаем основную информацию по квартирам

Объединяем всё в одну функцию! 

In [7]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(get_general_information(soup))
    infa_dict.update(get_about(soup))
    infa_dict.update(get_price(soup))
    infa_dict.update(get_description(soup))
    infa_dict.update(get_square_info(soup))
    infa_dict.update(get_address(soup))
    infa_dict.update(get_metro(soup))
    infa_dict.update(get_title(soup))
    infa_dict.update(get_coord(soup))
    infa_dict.update(get_reinovation(soup))
    return infa_dict

Основной цикл: 

In [9]:
all_infa = [ ] # для данных

In [ ]:
for url in tqdm_notebook(hrefs_total[27714:30000]):
    print(url)
    
    try:
        time.sleep(1)
        soup = get_soup(url)
        
    except ConnectionError:
        print('конэкшон ерор')
        time.sleep(60)
        soup = get_soup(url)
    if soup == None:
        continue
        
    # пока вылезает капча, отдыхаем
    while is_capcha_or_badip(soup):
        time.sleep(60)            # сплю N секунд
        soup = get_soup(url)     # внеочередная попытка пробится сквозь капчу 
            
    if soup.find("div", {"data-name": "OfferUnpublished"}):
        print('объявление снято')
        continue
        
    cur_infa = infa_download(soup)
    cur_infa['href'] = url
    all_infa.append(cur_infa)

https://www.cian.ru/sale/flat/225721370/
Капча вылезла :(
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/227687804/
объявление снято
https://www.cian.ru/sale/flat/226082147/
https://www.cian.ru/sale/flat/208761961/
https://www.cian.ru/sale/flat/226588862/
Капча вылезла :(
https://www.cian.ru/sale/flat/223301709/
объявление снято
https://www.cian.ru/sale/flat/227059703/
объявление снято
https://www.cian.ru/sale/flat/224815852/
Подозрительная сеть :(
Капча вылезла :(
https://www.cian.ru/sale/flat/227640836/
объявление снято
https://www.cian.ru/sale/flat/221979875/
https://www.cian.ru/sale/flat/222808033/
https://www.cian.ru/sale/flat/224099919/
https://www.cian.ru/sale/flat/190149401/
https://www.cian.ru/sale/flat/227498654/
объявление снято
https://www.cian.ru/sale/flat/213064876/
объявление снято
https://www.cian.ru/sale/flat/226276515/
конэкшон ерор
https://www.cian.ru/sale/flat/225237713/
объявление снято
https://www.cian.ru/sale/flat/227579902/
https://www.cian.ru/s

https://www.cian.ru/sale/flat/227661234/
https://www.cian.ru/sale/flat/223322715/
объявление снято
https://www.cian.ru/sale/flat/188561743/
объявление снято
https://www.cian.ru/sale/flat/200289730/
https://www.cian.ru/sale/flat/221684040/
https://www.cian.ru/sale/flat/227259730/
объявление снято
https://www.cian.ru/sale/flat/219929524/
https://www.cian.ru/sale/flat/226618626/
объявление снято
https://www.cian.ru/sale/flat/226547446/
https://www.cian.ru/sale/flat/225339121/
https://www.cian.ru/sale/flat/217958738/
Капча вылезла :(
Капча вылезла :(
Подозрительная сеть :(
https://www.cian.ru/sale/flat/223576442/
https://www.cian.ru/sale/flat/224491132/
https://www.cian.ru/sale/flat/224186617/
https://www.cian.ru/sale/flat/225676843/
Капча вылезла :(
Подозрительная сеть :(
Подозрительная сеть :(
Капча вылезла :(
Капча вылезла :(
объявление снято
https://www.cian.ru/sale/flat/223812711/
https://www.cian.ru/sale/flat/227053939/
объявление снято
https://www.cian.ru/sale/flat/226286198/
Капча 

https://www.cian.ru/sale/flat/164301683/
Капча вылезла :(
Капча вылезла :(
https://www.cian.ru/sale/flat/208691102/
https://www.cian.ru/sale/flat/224182468/
https://www.cian.ru/sale/flat/226049071/


In [12]:
hrefs_total.index(url)  # чтобы понимать откуда продолжать скачку, если код упал

27714

In [13]:
len(all_infa)

313

Сохранялка. Сохраняем в формате `info_2000_5000.pickle`, где цифры означают с какого по какой файл скачали.

In [14]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 
with open('data/info_27183_27714.pickle', 'wb') as f:
    pickle.dump(all_infa, f)

In [ ]:
# можно проверить всё ли ок сохранилось
with open('data/info_25_30.pickle', 'rb') as f:
    data_new = pickle.load(f)

Поглазеть на данные: 

In [1]:
import pandas as pd
df = pd.DataFrame(all_infa)
df.head()

NameError: name 'all_infa' is not defined

In [68]:
df.shape

(1295, 35)

In [69]:
df.isnull().sum()

href                     0
lat                      0
long                     0
Аварийность            402
Балкон/лоджия          550
Вид из окон            394
Высота потолков        630
Газоснабжение         1016
Год постройки          411
Жилая                  256
Кухня                  274
Лифты                   50
Мусоропровод           744
Общая                    0
Отделка               1105
Отопление              489
Парковка               672
Планировка             802
Площадь комнат         454
Подъезды               440
Построен               645
Ремонт                 405
Санузел                174
Срок сдачи            1026
Строительная серия    1000
Тип дома               354
Тип жилья                0
Тип перекрытий         499
Этаж                     0
адрес                    0
комментарий           1274
метро                    0
описание                 0
тайтл                    0
цена                     0
dtype: int64